# Kntu AI Final project 4021

### Import the nessecarry libraries

In [41]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

### Preprocessing the data

In [42]:
df = pd.read_csv("dataset.csv")
df.head()

,comment,sentiment
0,"Oh my god, it just doesn't get any worse than ...",negative
1,If you're a layman interested in quantum theor...,negative
2,It's amazing that this no talent actor Chapa g...,negative
3,This must be one of the most overrated Spanish...,negative
4,Some critics have compared Chop Shop with the ...,positive


The data is balanced so we can continue our journey

In [43]:
df.describe()

,comment,sentiment
count,45000,45000
unique,44664,2
top,"Hilarious, clean, light-hearted, and quote-wor...",positive
freq,4,22501


We one hot encode the sentiment column science it is categorical

In [44]:
one_hot_encoder = OneHotEncoder(sparse=False)
encoded_data = one_hot_encoder.fit_transform(df[['sentiment']])
one_hot_encoder.categories_

d:\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:808: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


[array(['negative', 'positive'], dtype=object)]

In [45]:
one_hot_df = pd.DataFrame(encoded_data, columns=one_hot_encoder.get_feature_names_out(['sentiment']))

In [46]:
df = pd.concat([df, one_hot_df], axis=1)
df = df.drop(['sentiment'], axis=1)
df.head()

,comment,sentiment_negative,sentiment_positive
0,"Oh my god, it just doesn't get any worse than ...",1.0,0.0
1,If you're a layman interested in quantum theor...,1.0,0.0
2,It's amazing that this no talent actor Chapa g...,1.0,0.0
3,This must be one of the most overrated Spanish...,1.0,0.0
4,Some critics have compared Chop Shop with the ...,0.0,1.0
